In [1]:
pipelines


{'__default__': Pipeline([
Node(standardize_m3, 'm3_stock_dataset', 'm3_stock_parquet', 'standardize_m3'),
Node(standardize_reflex, 'reflex_stock_dataset', 'reflex_stock_parquet', 'standardize_reflex'),
Node(map_m3, {'m3_df': 'm3_stock_parquet', 'rules': 'params:m3_mapping_rules', 'pos_df': 'm3_po_dataset'}, 'm3_map', 'map_m3'),
Node(map_reflex, {'reflex_df': 'reflex_stock_parquet', 'mapping': 'params:reflex_mapping_rules'}, 'reflex_map', 'map_reflex_category'),
Node(build_reflex_m3_wide_with_lotless, {'reflex_map': 'reflex_map', 'm3_map': 'm3_map', 'depots': 'params:m3_depots_columns'}, 'corr_dataset', 'build_reflex_m3_wide'),
Node(compute_m3_reliquat, {'m3_map': 'm3_map', 'reflex_map': 'reflex_map'}, 'm3_reliquat', 'compute_m3_reliquat_node')
]), 'preprocessing': Pipeline([
Node(map_m3, {'m3_df': 'm3_stock_parquet', 'rules': 'params:m3_mapping_rules', 'pos_df': 'm3_po_dataset'}, 'm3_map', 'map_m3'),
Node(map_reflex, {'reflex_df': 'reflex_stock_parquet', 'mapping': 'params:reflex_map

In [2]:
catalog


'm3_stock_dataset': kedro_datasets.pandas.sql_dataset.SQLQueryDataset
'm3_po_dataset': kedro_datasets.pandas.sql_dataset.SQLQueryDataset
'reflex_stock_dataset': kedro_datasets.pandas.sql_dataset.SQLQueryDataset
'm3_stock_parquet': kedro_datasets.pandas.parquet_dataset.ParquetDataset
'reflex_stock_parquet': kedro_datasets.pandas.parquet_dataset.ParquetDataset
'm3_map': kedro_datasets.pandas.parquet_dataset.ParquetDataset
'reflex_map': kedro_datasets.pandas.parquet_dataset.ParquetDataset
'm3_reliquat': kedro_datasets.pandas.parquet_dataset.ParquetDataset
'corr_dataset': kedro_datasets.pandas.parquet_dataset.ParquetDataset
'reflex_m3_regul': kedro_datasets.pandas.parquet_dataset.ParquetDataset
'stock_m3_rfx': kedro_datasets.pandas.csv_dataset.CSVDataset
'parameters': kedro.io.memory_dataset.MemoryDataset(data='<dict>')
'params:parquet_compression': kedro.io.memory_dataset.MemoryDataset(data='<str>')
'params:reflex_mapping_rules': kedro.io.memory_dataset.MemoryDataset(data='<dict>')
'para

## Analyse des cas 

### Cas listés 
https://wolfsa.sharepoint.com/sites/ERP/Documents%20partages/General/20%20Implementation/40%20Go%20live/Stocks-WMS/STOCK_251208/Base%20de%20travail%20stocks%20RFX%20vs%20M3%20NEW_251108.xlsx?web=1

In [28]:
# dataFrame de vérification
corr = catalog.load('corr_dataset')
reliq = catalog.load('m3_reliquat')

[12/22/25 14:47:24] INFO     Loading data from corr_dataset (ParquetDataset)...                ]8;id=129272;file:///home/mdelapintiere/ds/regulstock/.venv/lib/python3.12/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=455742;file:///home/mdelapintiere/ds/regulstock/.venv/lib/python3.12/site-packages/kedro/io/data_catalog.py#1048\1048]8;;\

                    INFO     Loading data from m3_reliquat (ParquetDataset)...                 ]8;id=521464;file:///home/mdelapintiere/ds/regulstock/.venv/lib/python3.12/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=388261;file:///home/mdelapintiere/ds/regulstock/.venv/lib/python3.12/site-packages/kedro/io/data_catalog.py#1048\1048]8;;\

In [4]:
m3 = catalog.load('m3_map')
reflex = catalog.load('reflex_map')
depots = ["100", "150", "200", "400"]

                    INFO     Loading data from m3_map (ParquetDataset)...                      ]8;id=886605;file:///home/mdelapintiere/ds/regulstock/.venv/lib/python3.12/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=201163;file:///home/mdelapintiere/ds/regulstock/.venv/lib/python3.12/site-packages/kedro/io/data_catalog.py#1048\1048]8;;\

                    INFO     Loading data from reflex_map (ParquetDataset)...                  ]8;id=932772;file:///home/mdelapintiere/ds/regulstock/.venv/lib/python3.12/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=917549;file:///home/mdelapintiere/ds/regulstock/.venv/lib/python3.12/site-packages/kedro/io/data_catalog.py#1048\1048]8;;\

In [5]:
reflex_sr = reflex[reflex.qualite != "RET"]
m3_sr = m3[m3.category != "RETOUR"]

In [6]:
reflex_lot = reflex_sr[reflex_sr['lot'].notna()].copy()
m3_lot = m3_sr[m3_sr['lot'].notna()].copy

m3_groups = m3.groupby(["depot","lot", "category", "type", "sku"], dropna=False)["qty_m3"].sum().reset_index()
m3_pivot = m3_groups.pivot_table(
            index=["lot", "type", "sku", "category"],
            columns=["depot"],
            values="qty_m3",
            aggfunc="sum",
            fill_value=0,
        ).reset_index()
m3_pivot = m3_pivot.rename(
        columns={d: f"stock_{d}" for d in depots if d in m3_pivot.columns}
    )

In [7]:
wide_with_lot = reflex_lot.merge(
    m3_pivot,
    on=["category", "lot", "sku"],
    how="left",
)

In [8]:
wide_with_lot[wide_with_lot.stock_400 != 0]

,sku,lot,qualite,qty_reflex,category,type,stock_100,stock_150,stock_200,stock_400
10,SC90A564ECH0005,EAN-FRA,STD,23.0,STOCK,A06,0.0,0.0,0.0,23.0
18,709212JUX075DS,1000002516,STD,20.0,STOCK,A06,0.0,0.0,0.0,20.0
21,70AAL41GCC075DS,1000002857,STD,76.0,STOCK,A06,0.0,0.0,0.0,76.0
35,SC56LAG98ECH000G,SWF24-0028,STD,13.0,STOCK,A06,0.0,0.0,0.0,13.0
48,SC61PAG42ECH0003,EAN-FRA,STD,1.0,STOCK,A06,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...
19611,60XAL69JTL3840S,1000002450,STD,1.0,STOCK,A06,0.0,0.0,0.0,1.0
19627,60PAN77JQZ3840S,1000002017,STD,3.0,STOCK,A06,0.0,0.0,0.0,3.0
19631,BD73AAK87ECH0001,SWF23-0049,STD,11.0,STOCK,A06,0.0,0.0,0.0,11.0
19637,SC68PAI75ECH0002,EAN-FRA,STD,2.0,STOCK,A06,0.0,0.0,0.0,2.0


In [9]:
reflex_lotless = reflex[reflex['lot'].isna()].copy()
lotless_m3 = m3[m3['lot'].isna()].copy()

lotless_m3_groups = lotless_m3.groupby(["depot", "category", "type", "sku"], dropna=False)["qty_m3"].sum().reset_index()
lotless_m3_pivot = lotless_m3_groups.pivot_table(
            index=["type", "sku", "category"],
            columns=["depot"],
            values="qty_m3",
            aggfunc="sum",
            fill_value=0,
        ).reset_index()

In [10]:
wide_no_lot = reflex_lotless.merge(
    lotless_m3_pivot,
    on=["category", "sku"],
    how="left",
)

In [11]:
wide_no_lot

,sku,lot,qualite,qty_reflex,category,type,100,150
0,SCPREPAETIQ00AH,None,STD,187.0,STOCK,B02,187.0,0.0
1,DIBUSTE0002,None,STD,5.0,STOCK,B01,5.0,0.0
2,CAT000000000028,None,STD,115.0,STOCK,B01,115.0,0.0
3,DIV000000000456,None,STD,75.0,STOCK,B01,75.0,0.0
4,SCDIVERS003X,None,STD,17.0,STOCK,B01,17.0,0.0
...,...,...,...,...,...,...,...,...
252,PLMETI000008050,None,STD,250.0,STOCK,B02,250.0,0.0
253,BDDISPLAY002G,None,STD,4.0,STOCK,B01,4.0,0.0
254,PLMBOI000007936,None,STD,3956.0,STOCK,B02,3977.0,0.0
255,SCPREPAETIQ0055,None,STD,4360.0,STOCK,B02,4360.0,0.0


Reste à faire

### Gestion des Lots existants dans M3 mais absents de Reflex

Si le lot est absent dans Reflex, il ne redescendra pas dans les tables précédentes. 

In [12]:
lot_list = []

for i in m3_sr.lot.unique():
    if i not in reflex_sr.lot.unique():
        lot_list.append(i)

In [13]:
print(lot_list)

['OI0500042021', 'OI0500066358', 'OI0500066054', 'OI0500073223', 'OI0500073224', 'OI0500062460', 'OI0500093635', 'OI0500042023', 'WF24-0006', 'OI0500096782', 'OI0500098814', 'OI0500076766', 'OI0500059332', '1000002523', 'WF24-0008', 'OI0500104645', 'OI0500055378', 'OI0500104599', 'OI0500057424', 'OI0500058258', 'OI0500040694', '1000002129', 'OI0500073232', 'OI0500066905', 'OI0500042009', '1000002077', 'OI0500095338', 'OI0500072331', 'OI0500038632', 'WF23-0133', 'OI0500098568', 'OI0500101092', '1000002376', 'WF23-0167', 'OI0500095320', 'OI0500097972', 'OI0500044598', 'WF23-0313', 'OI0500073234', 'OI0500073247', 'WF23-0367', 'OI0500067332', '99999', 'OI0500076787', 'OI0500039049', 'OI0500059341', 'WF23-0161', 'WF20-0011', 'OI0500059348', 'OI0500073237', 'OI0500095334', 'OI0500059533', 'OI0500061240', 'OI0500104646', 'OI0500073243', '1000003093', 'OI0500072337', 'WF24-0146', 'WF23-0363', 'OI0500042005', 'OI0500094798', 'OI0500057537', 'OI0500102657', 'OI0500072453', 'OI0500076762', '10000

In [14]:
wide_reliquat = reflex_lot.merge(
    m3_pivot,
    on=["category", "lot", "sku"],
    how="right",
)

In [15]:
wide_reliquat_no_lot = reflex_lotless.merge(
    lotless_m3_pivot,
    on=["category", "sku"],
    how="right",
)

In [16]:
wide_reliquat_no_lot = wide_reliquat_no_lot[wide_reliquat_no_lot.qualite.isna()]
wide_reliquat_no_lot

,sku,lot,qualite,qty_reflex,category,type,100,150
0,02XAO34GCA00TUX,NaN,NaN,NaN,STOCK,B01,196.0,10.0
1,02XAO34HCY00TUX,NaN,NaN,NaN,STOCK,B01,128.0,3.0
4,86XAP23JYH00TUX,NaN,NaN,NaN,NDISP,B01,1.0,0.0
6,86XAP26JIQ00TUX,NaN,NaN,NaN,NDISP,B01,2.0,0.0
30,CAT000000000041,NaN,NaN,NaN,STOCK,B01,3.0,0.0
128,SCDIVERS006U,NaN,NaN,NaN,NDISP,B01,4.0,0.0
167,PLMBOI000007717,NaN,NaN,NaN,NDISP,B02,100.0,0.0
168,PLMBOI000007718,NaN,NaN,NaN,NDISP,B02,100.0,0.0
222,SCCARTON0001,NaN,NaN,NaN,STOCK,B02,12.0,0.0
251,CAT000000000048,NaN,NaN,NaN,NDISP,None,24000.0,0.0


In [17]:
wide_reliquat = wide_reliquat[wide_reliquat.qualite.isna()]
wide_reliquat

,sku,lot,qualite,qty_reflex,category,type,stock_100,stock_150,stock_200,stock_400
1,BD60XAL13ECH0001,0000,NaN,NaN,NDISP,A06,0.0,0.0,0.0,12.0
3,70564OR100D,10000003095,NaN,NaN,STOCK,A01,10.0,0.0,0.0,0.0
20,SC79SAE46000A,1000001950,NaN,NaN,STOCK,A01,0.0,10.0,0.0,0.0
22,SC79SAE46000G,1000001950,NaN,NaN,STOCK,A01,0.0,10.0,0.0,0.0
23,SC79SAE46000L,1000001950,NaN,NaN,STOCK,A01,0.0,5.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
22522,SC70A5640531,WF24-0168,NaN,NaN,NDISP,A01,1.0,0.0,0.0,0.0
22524,SC61P564013M,WF24-0168,NaN,NaN,NDISP,A01,1.0,0.0,0.0,0.0
22527,SC70A5640523,WF24-0168,NaN,NaN,NDISP,A01,1.0,0.0,0.0,0.0
22532,SC70A564053V,WF24-0168,NaN,NaN,NDISP,A01,1.0,0.0,0.0,0.0


### Contrôles manuels

In [18]:
lot_cible = "WF22-0213"
sku_cible = "SC70UAJ27000W"

#### LOTS

In [19]:
wide_reliquat[wide_reliquat.lot == lot_cible]

,sku,lot,qualite,qty_reflex,category,type,stock_100,stock_150,stock_200,stock_400
16937,SC70UAJ270016,WF22-0213,NaN,NaN,NDISP,A01,1.0,0.0,0.0,0.0
16949,SC40XAJ270004,WF22-0213,NaN,NaN,STOCK,A01,0.0,2.0,0.0,0.0
16950,SC40XAJ270006,WF22-0213,NaN,NaN,STOCK,A01,0.0,3.0,0.0,0.0
16955,SC70UAJ27000W,WF22-0213,NaN,NaN,DES,A01,0.0,0.0,1.0,0.0
16956,SC70UAJ27000Z,WF22-0213,NaN,NaN,STOCK,A01,1.0,1.0,0.0,0.0


In [20]:
corr[corr.lot == lot_cible]

,sku,lot,qualite,type,category,qty_reflex,stock_100,stock_200,stock_150,stock_400,stock_total_m3,ecart_rfx_m3
659,SC70UAJ270016,WF22-0213,RET,A01,NDISP,1.0,1.0,0.0,0.0,0.0,1.0,0.0
961,SC70UAJ27000D,WF22-0213,STD,A01,STOCK,1.0,1.0,0.0,1.0,0.0,2.0,-1.0
999,SC70UAJ270001,WF22-0213,STD,A01,STOCK,1.0,1.0,0.0,0.0,0.0,1.0,0.0
1850,SC70UAJ270012,WF22-0213,STD,A01,STOCK,1.0,1.0,0.0,1.0,0.0,2.0,-1.0
2806,SC70UAJ270016,WF22-0213,STD,A01,STOCK,1.0,0.0,0.0,1.0,0.0,1.0,0.0
3334,SC70UAJ27000O,WF22-0213,STD,A01,STOCK,2.0,2.0,0.0,1.0,0.0,3.0,-1.0
4512,SC70UAJ270011,WF22-0213,STD,A01,STOCK,1.0,1.0,0.0,0.0,0.0,1.0,0.0
4883,SC40XAJ270002,WF22-0213,STD,A01,STOCK,1.0,1.0,0.0,0.0,0.0,1.0,0.0
6031,SC31XAJ270002,WF22-0213,STD,A01,STOCK,2.0,0.0,0.0,1.0,0.0,1.0,1.0
6477,SC40XAJ270003,WF22-0213,STD,A01,STOCK,2.0,1.0,0.0,0.0,0.0,1.0,1.0


In [21]:
m3[m3.lot == lot_cible]

,sku,sku_m3,lot,depot,category,type,qty_m3,is_sms,is_150
1352,SC31XAJ270008,31XAJ27IQG3436,WF22-0213,100,STOCK,A01,1.0,0,0
1356,SC31XAJ270002,31XAJ27IQG4244,WF22-0213,150,STOCK,A01,1.0,0,0
1358,SC31XAJ270006,31XAJ27IQG4648,WF22-0213,100,STOCK,A01,41.0,0,0
1363,SC31XAJ270007,31XAJ27IQH4648,WF22-0213,100,STOCK,A01,148.0,0,0
1365,SC31XAJ270007,31XAJ27IQH4648,WF22-0213,150,STOCK,A01,15.0,0,0
1763,SC31XAJ270004,31XAJ27IQH4244,WF22-0213,100,STOCK,A01,2.0,0,0
2213,SC40XAJ270001,40XAJ27IQG4648,WF22-0213,100,STOCK,A01,2.0,0,0
2215,SC40XAJ270001,40XAJ27IQG4648,WF22-0213,150,STOCK,A01,1.0,0,0
2219,SC40XAJ270004,40XAJ27IQH5052,WF22-0213,150,STOCK,A01,2.0,0,0
2605,SC40XAJ270003,40XAJ27IQG4244,WF22-0213,100,STOCK,A01,1.0,0,0


In [22]:
reflex[reflex.lot == lot_cible]

,sku,lot,qualite,qty_reflex,category
668,SC70UAJ270016,WF22-0213,RET,1.0,NDISP
976,SC70UAJ27000D,WF22-0213,STD,1.0,STOCK
1014,SC70UAJ270001,WF22-0213,STD,1.0,STOCK
1875,SC70UAJ270012,WF22-0213,STD,1.0,STOCK
2842,SC70UAJ270016,WF22-0213,STD,1.0,STOCK
3373,SC70UAJ27000O,WF22-0213,STD,2.0,STOCK
4572,SC70UAJ270011,WF22-0213,STD,1.0,STOCK
4947,SC40XAJ270002,WF22-0213,STD,1.0,STOCK
6102,SC31XAJ270002,WF22-0213,STD,2.0,STOCK
6552,SC40XAJ270003,WF22-0213,STD,2.0,STOCK


#### SKU

In [23]:
wide_reliquat[wide_reliquat.sku == sku_cible]

,sku,lot,qualite,qty_reflex,category,type,stock_100,stock_150,stock_200,stock_400
16955,SC70UAJ27000W,WF22-0213,NaN,NaN,DES,A01,0.0,0.0,1.0,0.0


In [24]:
wide_reliquat_no_lot[wide_reliquat_no_lot.sku == sku_cible]

,sku,lot,qualite,qty_reflex,category,type,100,150


In [25]:
corr[corr.sku == sku_cible]

,sku,lot,qualite,type,category,qty_reflex,stock_100,stock_200,stock_150,stock_400,stock_total_m3,ecart_rfx_m3
3691,SC70UAJ27000W,WF23-0017,DES,A01,DES,1.0,0.0,1.0,0.0,0.0,1.0,0.0
12018,SC70UAJ27000W,WF23-0289,STD,A01,STOCK,100.0,89.0,0.0,11.0,0.0,100.0,0.0
15485,SC70UAJ27000W,WF23-0108,STD,A01,STOCK,4.0,0.0,0.0,4.0,0.0,4.0,0.0


In [26]:
m3[m3.sku == sku_cible]

,sku,sku_m3,lot,depot,category,type,qty_m3,is_sms,is_150
18972,SC70UAJ27000W,70UAJ27IQH095E,WF23-0289,100,STOCK,A01,89.0,0,0
18973,SC70UAJ27000W,70UAJ27IQH095E,WF23-0108,150,STOCK,A01,4.0,0,0
18974,SC70UAJ27000W,70UAJ27IQH095E,WF23-0289,150,STOCK,A01,11.0,0,0
20363,SC70UAJ27000W,70UAJ27IQH095E,WF22-0213,200,DES,A01,1.0,0,0
20364,SC70UAJ27000W,70UAJ27IQH095E,WF23-0017,200,DES,A01,1.0,0,0


In [27]:
reflex[reflex.sku == sku_cible]

,sku,lot,qualite,qty_reflex,category
3736,SC70UAJ27000W,WF23-0017,DES,1.0,DES
12163,SC70UAJ27000W,WF23-0289,STD,100.0,STOCK
15667,SC70UAJ27000W,WF23-0108,STD,4.0,STOCK
